In [ ]:
import os
import pandas as pd
from tqdm import tqdm
# os.chdir('C:/Users/jinyo/Downloads/')

In [ ]:
sentiment_dict = pd.read_csv('sentiment dictionary.csv', index_col=0)
sentiment_dict

,pos,mid,neg
0,방긋,아직,회의적
1,상회,보통,바닥
2,신선,vs,이탈
3,신박,중립,떨어지
4,투혼,관망,안좋게
...,...,...,...
954,NaN,NaN,어처구니
955,NaN,NaN,멍충
956,NaN,NaN,얄짤
957,NaN,NaN,전멸


In [ ]:
pos_li = sentiment_dict['pos'].dropna().values
mid_li = sentiment_dict['mid'].dropna().values
neg_li = sentiment_dict['neg'].dropna().values

## vocab 불러오기

### LSTM vocab

In [ ]:
vocab_LSTM = pd.read_csv('[Model2] vocab_score.csv', header = None)
vocab_LSTM.columns = ['word', 'score']
vocab_LSTM

,word,score
0,화신,-2.579001
1,신고,243.419583
2,경신,203.355900
3,단기,79.481405
4,중기,65.175381
...,...,...
35818,윤찍들,-0.522020
35819,돈벌,-0.522020
35820,사업수익,-0.522020
35821,반원,-0.522020


### BERT vocab

## SP기반 Vocab을 제작한 감성사전과 matching하여 새로운 감성사전 구축

In [ ]:
def sentimental_rescoring(df):
    # 입력받은 데이터프레임 복사 및 컬럼 추가
    df_result = df.copy()
    df_result['Pos'] = 0
    df_result['Neg'] = 0
    df_result['Mid'] = 0
    
    # 감성 지수는 긍정 : 1, 중립 : 0, 부정 : -1, 해당 데이터 제외 : 999
    df_result['new감성지수'] = 999    
    
    # 감성 사전에 따른 텍스트 검출
    print('긍정 단어 검색중')
    for pos in tqdm(pos_li) :
        str_expr = f"word.str.contains('{pos}')"
        df_result.loc[df_result.query(str_expr).index, 'Pos'] = 1
    
    print('부정 단어 검색중')
    for neg in tqdm(neg_li) :
        str_expr = f"word.str.contains('{neg}')"
        df_result.loc[df_result.query(str_expr).index, 'Neg'] = 1
    
    print('중립 단어 검색중')
    for mid in tqdm(mid_li) :
        str_expr = f"word.str.contains('{mid}')"
        df_result.loc[df_result.query(str_expr).index, 'Mid'] = 1
    
    # 모든 종류의 단어가 검출 되면 제외
    df_result.loc[df_result.query('Pos == 1 and Neg == 1 and Mid == 1').index, 'new감성지수'] = 999
    
    # 중립 단어가 검출되면 중립
    df_result.loc[df_result.query('Mid == 1').index, 'new감성지수'] = 0
    
    # 긍정 단어만이 검출되면 긍정
    df_result.loc[df_result.query('Pos == 1 and Neg == 0 and Mid == 0').index, 'new감성지수'] = 1
    
    # 부정 단어만이 검출되면 부정
    df_result.loc[df_result.query('Pos == 0 and Neg == 1 and Mid == 0').index, 'new감성지수'] = -1

    return df_result

In [ ]:
df = sentimental_rescoring(vocab_LSTM)

긍정 단어 검색중


100%|████████████████████████████████████████████████████████████████████████████████| 641/641 [00:11<00:00, 56.00it/s]


부정 단어 검색중


100%|████████████████████████████████████████████████████████████████████████████████| 959/959 [00:16<00:00, 56.67it/s]


중립 단어 검색중


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [00:01<00:00, 50.00it/s]


In [ ]:
df.to_csv('vocab df for sentiment_dictionary_new.csv', encoding = 'utf-8-sig')